In [3]:
import torch
import torch.nn.functional as F
import numpy as np
import Preprocessor as pp
import sounddevice as sd

In [7]:
sd.default.device = (0, None)
print(sd.query_devices())

> 0 ABKO MP3300: USB Audio (hw:2,0), ALSA (1 in, 0 out)
  1 pulse, ALSA (32 in, 32 out)
< 2 default, ALSA (32 in, 32 out)


In [1]:
MODEL_PATH = "model/resnet50.hef"
CLASS_NAME = ["danger", "fire", "gas", "non", "tsunami"]

# 하이퍼 파라미터
CLASS_NUM = len(CLASS_NAME)
SAMPLE_RATE = 16000
DURATION = 1
NUM_SAMPLES = SAMPLE_RATE * DURATION
BATCH_SIZE = 32
LEARNING_RATE = 1e-4

In [ ]:
def logmel_transform(audio_np):
    # numpy 배열을 torch 텐서로 변환, 배치 차원 추가: (1, num_samples)
    waveform = torch.tensor(audio_np, dtype=torch.float32)

    # 2채널 이상일때 1채널로 변환
    if waveform.shape[0] > 1:
        waveform = waveform.mean(axis=-1)
    
    # 오디오 샘플의 길이조정
    if waveform.shape[1] < NUM_SAMPLES: # 길이가 부족하면 0(무음)을 채워 길이를 연장
        waveform = F.pad(waveform, (0, NUM_SAMPLES - waveform.shape[1])) # num_samples와 현재의 길이의 차 만큼 0을 패딩
    else:
        waveform = waveform[:, :NUM_SAMPLES] # 길이가 길면 슬라이싱

    # 절댓값 정규화(-1 ~ 1)
    waveform = waveform / (waveform.abs().max() + 1e-9)

    logmel = pp.logmel(waveform).unsqueeze(0) # [batch, channel, mel_bins, time]

    return logmel


def predict(model, logmel):
    logmel = logmel

    with torch.no_grad():
        outputs = model(logmel)
        probabilities = F.softmax(outputs, dim=1).cpu().numpy()[0]
        idx = probabilities.argmax()

    return idx, probabilities

In [9]:
print("실시간 마이크 입력을 시작합니다.")

try:
    # 실시간 마이크 입력 루프
    while True:
        # 오디오를 실시간으로 마이크에서 녹음
        audio_np = sd.rec(frames=NUM_SAMPLES, # 녹음할 샘플 수 (1초 분량)
                          samplerate=SAMPLE_RATE, # 샘플링 레이트
                          channels=1, # 단일 채널 (모노)
                          dtype="float32" # float32 타입으로 녹음
                          ).transpose(1, 0)
        sd.wait() # 녹음 완료까지 대기

        # audio_np = logmel(audio_np)
        logmel = logmel_transform(audio_np)
        idx, probabilities = predict(model, logmel)

        print(f"{CLASS_NAME[idx]} | {probabilities[idx] * 100:.2f}%")

except KeyboardInterrupt as e:
    print("\n실시간 마이크 입력을 종료합니다.\n", e)

실시간 마이크 입력을 시작합니다.
non | 98.24%
non | 99.79%
non | 98.75%
non | 99.58%
non | 99.93%
danger | 46.62%
danger | 96.83%
fire | 74.03%
danger | 60.38%
danger | 45.18%
danger | 99.85%
fire | 45.77%
non | 99.17%
non | 95.58%
fire | 95.10%
fire | 94.98%
fire | 99.20%
fire | 99.84%
fire | 99.94%
non | 96.89%
non | 99.85%
non | 99.76%
non | 94.20%
non | 98.77%
non | 99.91%
non | 99.96%
non | 99.73%
non | 99.64%
non | 99.60%
non | 98.47%
non | 99.93%
non | 99.72%
non | 63.02%
fire | 82.75%
fire | 45.19%
fire | 99.19%
fire | 99.91%
non | 99.28%
fire | 99.84%
fire | 99.78%
fire | 98.45%
non | 99.21%
non | 99.70%
non | 99.87%
non | 99.41%
non | 97.27%
non | 99.70%
fire | 99.70%
fire | 90.02%
fire | 99.73%
non | 99.76%
fire | 94.19%
fire | 88.55%
fire | 84.69%
non | 99.70%
fire | 100.00%
fire | 99.99%
fire | 99.99%
fire | 99.96%
fire | 99.92%
non | 99.70%
non | 98.02%
non | 77.18%
tsunami | 96.54%
fire | 91.23%
fire | 61.19%
fire | 87.93%
non | 99.64%
danger | 99.18%
danger | 99.94%
non | 99.85%
dang